In [69]:
import requests
import json
import sys
import os
import pandas as pd
import numpy as np
from fuzzywuzzy import process

def get_departures(stop_id):
    """Abfahrtszeiten für eine bestimmte Haltestelle abrufen"""
    url = f"http://www.wienerlinien.at/ogd_realtime/monitor?stopId={stop_id}"
    
    response = requests.get(url)
    data = response.json()
    
    if "data" in data and "monitors" in data["data"]:
        return data["data"]["monitors"]
    return None

def print_departures(stop_id):
    """Abfahrtszeiten formatiert ausgeben"""
    departures = get_departures(stop_id)
    
    if not departures:
        print("Keine Daten gefunden")
        return None, [], []
    
    countdown_list = []
    departure_info = []  # List to store structured departure information
    line_data_full = []

    for monitor in departures:
        location = monitor["locationStop"]["properties"]["title"]
        
        for line in monitor["lines"]:
            line_data_full.append(line)
            line_name = line["name"]
            towards = line["towards"]
            
            if "departures" in line and "departure" in line["departures"]:
                for departure in line["departures"]["departure"]:
                    countdown = departure["departureTime"]["countdown"]
                    countdown_list.append(countdown)
                    # Store structured info with line name and direction
                    departure_info.append({
                        'line': line_name,
                        'direction': towards,
                        'countdown': countdown
                    })
    
    # Create formatted strings with line name, direction and countdown
    formatted_departures = [
        f"Linie: {info['line']} - Richtung: {info['direction']} - Abfahrt in {info['countdown']} Minuten"
        for info in departure_info
    ]
    
    return countdown_list, formatted_departures, line_data_full

def fuzzy_search_best_match(search_term, df, column_name="StopText", threshold=50):
    """
    Führt eine Fuzzy-Suche durch und gibt nur die beste Übereinstimmung zurück
    
    Parameters:
    -----------
    search_term : str
        Der Suchbegriff
    df : pandas.DataFrame
        Der Datensatz mit den Haltestellen
    column_name : str
        Die Spalte mit den Haltestellennamen
    threshold : int
        Minimaler Ähnlichkeitswert (0-100)
        
    Returns:
    --------
    pandas.Series oder None
        Die beste Übereinstimmung oder None wenn keine gefunden wurde
    """
    # Eindeutige Haltestellennamen bekommen
    unique_stations = df[column_name].unique()
    
    # Fuzzy-Suche für die beste Übereinstimmung
    match = process.extractOne(search_term, unique_stations, score_cutoff=threshold)
    
    if not match:
        return None
    
    # Die beste Übereinstimmung zurückgeben
    station_name, score = match
    best_match = df[df[column_name] == station_name].copy()
    best_match['similarity_score'] = score
    
    # Nur die erste Zeile zurückgeben, falls mehrere mit dem gleichen Namen
    bestmatch_name = best_match["StopText"].values[0]
    best_match = best_match[best_match["StopText"] == bestmatch_name]
    
    return best_match

# Hauptprogramm
haltestellen = pd.read_csv('data/wienerlinien-ogd-haltepunkte.csv', sep=';')

# Hier beginnt die Fuzzy-Suche
haltestellen_name = "Dresdner"  # Zum Beispiel "Kutsch"

# Beste Übereinstimmung finden
best_match = fuzzy_search_best_match(haltestellen_name, haltestellen)

if best_match is None:
    print(f"Keine Haltestelle gefunden für: '{haltestellen_name}'")
else:
    # Informationen zur besten Übereinstimmung
    
    # Original-Format der Ausgabe für die Abfahrtszeiten
    for index, row in best_match.iterrows():
        if pd.notna(row["DIVA"]):  # Besser als np.isnan für pandas
            stop_id = row["StopID"]
            stop_name = row["StopText"]
            print(f"Haltestelle: {stop_name} (ID: {stop_id})")
            countdown, formatted_departures, line_data_full = print_departures(stop_id)
            print("-" * 40)
            
            # Ausgabe der Abfahrtszeiten mit Liniennamen und Richtung
            if formatted_departures:
                for departure in formatted_departures:
                    print(departure)
            else:
                print("Keine Abfahrten gefunden.")
            
            print("-" * 40)

Haltestelle: Dresdner Straße U (ID: 318)
----------------------------------------
Linie: 2 - Richtung: Friedrich-Engels-Platz - Abfahrt in 20 Minuten
Linie: 2 - Richtung: Friedrich-Engels-Platz - Abfahrt in 28 Minuten
Linie: 2 - Richtung: Friedrich-Engels-Platz - Abfahrt in 43 Minuten
Linie: 37A - Richtung: Nußdorfer Straße U - Abfahrt in 12 Minuten
Linie: N29 - Richtung: Floridsdorf U - Abfahrt in 52 Minuten
----------------------------------------
Haltestelle: Dresdner Straße U (ID: 323)
----------------------------------------
Linie: N29 - Richtung: Wittelsbachstraße - Abfahrt in 68 Minuten
----------------------------------------
Haltestelle: Dresdner Straße U (ID: 2951)
----------------------------------------
Linie: 5A - Richtung: Griegstraße - Abfahrt in 5 Minuten
----------------------------------------
Haltestelle: Dresdner Straße U (ID: 2957)
Keine Daten gefunden
----------------------------------------
Keine Abfahrten gefunden.
----------------------------------------


In [65]:
haltestellen

,StopID,DIVA,StopText,Municipality,MunicipalityID,Longitude,Latitude
0,2,60201421.0,Venediger Au,Wien,49000001.0,48.217909,16.396536
1,3,60201892.0,Messe Prater,Wien,49000001.0,48.217945,16.406633
2,4,60201009.0,Perspektivstraße,Wien,49000001.0,48.217969,16.401306
3,5,60200195.0,Börse,Wien,49000001.0,48.215784,16.364861
4,7,60201848.0,"Praterstern, Lassallestraße",Wien,49000001.0,48.219549,16.393769
...,...,...,...,...,...,...,...
5206,9405,NaN,Bahngasse/Kasematten,NaN,NaN,NaN,NaN
5207,9500,NaN,Rochusgasse (DUMMY Marathon),NaN,NaN,NaN,NaN
5208,9900,NaN,Salmannsdorf Dummy,NaN,NaN,NaN,NaN
5209,9989,NaN,"Bhf. Hetzendorf, Altmann",NaN,NaN,NaN,NaN


In [13]:
countdown_strings

['Abfahrt in 4 Minuten',
 'Abfahrt in 11 Minuten',
 'Abfahrt in 17 Minuten',
 'Abfahrt in 25 Minuten',
 'Abfahrt in 32 Minuten',
 'Abfahrt in 40 Minuten',
 'Abfahrt in 47 Minuten',
 'Abfahrt in 55 Minuten',
 'Abfahrt in 62 Minuten']